In [12]:
# load extensions
%reload_ext autoreload
%autoreload 1

%reload_ext watermark
%watermark -u -d -v -g -h -p pandas

last updated: 2016-08-30 

CPython 3.5.2
IPython 5.1.0

pandas 0.18.1
host name: dk50r3-3341mac4
Git hash: e41bf346ac4573753024da16fa914a14e3518f0f


In [62]:
# imports
import os
import gzip

import pandas as pd
from Bio import SeqIO
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist

In order to understand what Morgan did, I decided to try to run the roi calling using the SMRT Portal. 

https://github.com/PacificBiosciences/Bioinformatics-Training/wiki/SMRT-Analysis-Virtual-Machine-Install

Make sure to change the Vagrantfile machine allocations as follows (~line 10)

```conf
vb.customize ["modifyvm", :id, "--memory", "20048"]
vb.customize ["modifyvm", :id, "--cpus", "6"]
```

I only ran a single sample ARL0001_1. This takes a long time even using 6 cores and 20 gb of RAM. From inside the portal software I downloaded the FASTQ file for ROI. When comparing the first read of these files, there is a 3bp insertion in my file. I think it is safe to say that the files are similar, but there are distinct differences. Perhaps a setting/version or maybe the algorithm is not deterministic and will not reproduce the exact same reads each time. 

In [27]:
# Location of FATQ files on my local machine
MORGANFQ = '/Users/fearjm/storage/fastq/PacBio/Data_20160630/ARL0001_1/roi1/reads_of_insert.fastq.gz'
MYFQ = '/Users/fearjm/tmp/pacbio/me.fastq'

In [35]:
# get and hanlde for each
morgan = SeqIO.parse(gzip.open(MORGANFQ, 'rt', encoding='utf-8'), 'fastq')
me = SeqIO.parse(open(MYFQ, 'r'), 'fastq')

# Get the first read
curr_Morgan = next(morgan)
curr_Me = next(me)

# Make sure the ids are the same
assert(curr_Morgan.id == curr_Me.id)

In [68]:
# Do a quick and dirty local alignment
matrix = matlist.blosum62
gap_open = -10
gap_extend = -0.5
 
alns = pairwise2.align.globalds(str(curr_Morgan.seq), str(curr_Me.seq), matrix, gap_open, gap_extend)
 
top_aln = alns[0]
aln_morgan, aln_me, score, begin, end = top_aln

print(">Morgan\n" + aln_morgan + "\n" + ">Me\n" + aln_me)

>Morgan
AAGCAGTGGTATCAACGCAGAGTACTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT---CTTTTTTTTGAGGTTGGTCAATATTCATCATCGATTTATTACGACATTCCATCGCAGCGTCTTAGGTTAACAATAACATTCAGTGCAAAGGTTATGTTTAAGAAAAGATCCACGTTTGATTTATTATACTACATCCGTTGCTAAGAGTAGTTCGAATTCGGTTGATTTTCTAATTTTTGTGACGCTCGCAAAATGAAATAATAAATAACCAAAGACAGTTTCAAATTAAGAAGGCTAATATCAAATTAAGGCCGTGGACGTGTGTTAAGACAATTCACATTACAATCCGGCAACAATTATTGCTGGGTTCTCAACGGCAGCCTTTATTTTTCGCAGGAACAGGACCGCCTCACGTCCATCAATGATTCGGTGATCGTAGGTCAGGGCGATGTACATCATAGGACGTATCTTGACCTCTCCCTTGACAGCAATTGGCCGCTCAAAGATGCCGTGCATGCCGAGAATGGCACTCTGTGGCGGATTGATGATCGGCGTGCCCATCAGGGATCCGAACACTCCGCCATTGCTGATGGTGAAGGTGCCACCGTCCATGTCTTCCACGGTGATGGCATCCCGCCTGGCCTTATCGGCCAGACCGGCCAGTGCGATCTCGATGTCCGCATAGTTCATGCCCTCCACGTTGCGGATGACGGGTACCACCAATCCACGGGGCGTAGCCACGGCCACAGAGATGTCAACGTAGTCGCGATACACGATGTCCGTGCCATCGATGACGGCGTTCACTACGGGCTGATCCTGCAGCGCATAGGCACTAGCCTTGGTGAAGATCGACATGAATCCAAACTTAATACCATACTTCTTAGTGAACGCATCCAGATTCTGCTTTCGGAAGTCCATGGCATAGCTCATATCAACCTCGTTGAAAGTGGTGAGCATGGCACATGTGTTCTGGGCATCCTTCAAGCGGGCGGCA